In [21]:
import os
import pickle
import random
import socket
import struct

import networkx as nx
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from src.dataset.dataset_info import datasets
from src.graph.graph_construction.window_graph import create_weightless_window_graph
from src.graph.graph_measures import calculate_graph_measures
from src.graph.centralities.add_centralities import add_centralities, add_centralities_as_node_features


In [22]:
multi_class = True

use_port_in_address = False

generated_ips = False

# graph_type = "flow"
# graph_type = "window"
graph_type = "line"

window_size= 10000

sort_timestamp = False

# k_fold = None
# k_fold = 5

validation_size = 0.1
test_size = 0.1

use_node_features = True
cn_measures = ["betweenness", "degree", "pagerank", "closeness", "k_truss"]
# cn_measures = ["betweenness", "degree", "closeness"]

network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank', 'src_closeness', 'dst_closeness', 'src_k_truss', 'dst_k_truss']
# network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank']

In [23]:
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
# name = "cic_ids_2017_5_percent"
name = "cic_ids_2017"
# name = "cic_bot_iot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"

dataset = datasets[name]

In [24]:
g_type = ""
if graph_type == "flow":
    g_type = "flow"
elif graph_type == "line":
    g_type = f"line_graph_{window_size}"
elif graph_type == "window":
    g_type = f"window_graph_{window_size}"
    
if multi_class:
    g_type += "__multi_class"
    
if use_node_features:
    g_type += "__n_feats"
    
# if k_fold:
#     g_type += f"__{k_fold}_fold"
    
if use_port_in_address:
    g_type += "__ports"
    
if generated_ips:
    g_type += "__generated_ips"
    
if sort_timestamp:
    g_type += "__sorted"
else:
    g_type += "__unsorted"
    
folder_path = os.path.join("datasets",name, g_type)
# folder_path = f"datasets/{name}/{g_type}"
folder_path

'datasets\\cic_ids_2017\\line_graph_10000__multi_class__n_feats__ports__unsorted'

In [25]:
df = pd.read_parquet(dataset.path)

In [26]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
index,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443.0,192.168.10.5,54865.0,6.0,7/7/2017 3:30,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80.0,192.168.10.5,55054.0,6.0,7/7/2017 3:30,109.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80.0,192.168.10.5,55055.0,6.0,7/7/2017 3:30,52.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443.0,192.168.10.16,46236.0,6.0,7/7/2017 3:30,34.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443.0,192.168.10.5,54863.0,6.0,7/7/2017 3:30,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0


In [27]:
cols_to_norm = list(set(list(df.columns))  - set(list([dataset.label_col, dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))

In [28]:
df[dataset.label_col].value_counts()

Label
0    2265910
1     548968
Name: count, dtype: int64

In [29]:
if generated_ips:
    df[dataset.src_ip_col] = df[dataset.src_ip_col].apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

In [30]:
if sort_timestamp:
    df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)

In [31]:
if use_port_in_address:
    df[dataset.src_port_col] = df[dataset.src_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.src_ip_col] = df[dataset.src_ip_col] + ':' + df[dataset.src_port_col]

    df[dataset.dst_port_col] = df[dataset.dst_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.dst_ip_col] = df[dataset.dst_ip_col] + ':' + df[dataset.dst_port_col]

In [32]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
index,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165:443,443,192.168.10.5:54865,54865,6.0,7/7/2017 3:30,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216:80,80,192.168.10.5:55054,55054,6.0,7/7/2017 3:30,109.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216:80,80,192.168.10.5:55055,55055,6.0,7/7/2017 3:30,52.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25:443,443,192.168.10.16:46236,46236,6.0,7/7/2017 3:30,34.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102:443,443,192.168.10.5:54863,54863,6.0,7/7/2017 3:30,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0


In [33]:
if multi_class:
    y = df[dataset.class_num_col]
else:
    y = df[dataset.label_col]

if sort_timestamp:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size)
else:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size, random_state=13, stratify=y)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size, random_state=13, stratify=y_tr)

del df

In [34]:
if graph_type == "line" and use_node_features:
    add_centralities(df = X_train, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_val, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_test, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    cols_to_norm = list(set(cols_to_norm) | set(network_features))
    

constructing graph
calculating betweenness


In [ ]:
scaler = StandardScaler()

X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_train['h'] = X_train[ cols_to_norm ].values.tolist()

cols_to_drop = list(set(list(X_train.columns)) - set(list([dataset.label_col, dataset.src_ip_col, dataset.dst_ip_col, dataset.class_num_col, 'h'])))
X_train.drop(cols_to_drop, axis=1, inplace=True)

X_val[cols_to_norm] = scaler.transform(X_val[cols_to_norm])
X_val['h'] = X_val[ cols_to_norm ].values.tolist()
X_val.drop(cols_to_drop, axis=1, inplace=True)

X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test[ cols_to_norm ].values.tolist()
X_test.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
if graph_type == "window" or graph_type == "line":

    create_weightless_window_graph(
        df=X_train,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "training"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_val,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "validation"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_test,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "testing"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")

In [ ]:
if graph_type == "flow":
	os.makedirs(folder_path, exist_ok=True)
	print(f"==>> X_train.shape: {X_train.shape}")
	print(f"==>> X_val.shape: {X_val.shape}")
	print(f"==>> X_test.shape: {X_test.shape}")


==>> X_train.shape: (2280051, 5)
==>> X_val.shape: (253339, 5)
==>> X_test.shape: (281488, 5)


In [ ]:
if graph_type == "flow":
    graph_name = "training_graph"

    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> calculated degrees, in 0.08815010264515877 seconds
==>> graph_measures: {'number_of_nodes': 19025, 'number_of_edges': 2280051, 'max_degree': 796741, 'avg_degree': 239.68998685939553, 'density': 0.006299673750509765}


In [ ]:
if graph_type == "flow":
    graph_name = "validation_graph"

    G = nx.from_pandas_edgelist(X_val, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> calculated degrees, in 0.029383700340986252 seconds
==>> graph_measures: {'number_of_nodes': 12740, 'number_of_edges': 253339, 'max_degree': 89076, 'avg_degree': 39.770643642072216, 'density': 0.0015609798116835}


In [ ]:
if graph_type == "flow":
    graph_name = "testing_graph"
    
    G = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h', dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    # graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")
    
    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> calculated degrees, in 0.02684839814901352 seconds
==>> graph_measures: {'number_of_nodes': 13253, 'number_of_edges': 281488, 'max_degree': 98178, 'avg_degree': 42.47913679921527, 'density': 0.0016027443706314243}
